In [1]:
import os
import random
import numpy as np
import torch
import gpytorch
from matplotlib import pyplot as plt
%config InlineBackend.figure_gformat = "retina"
%matplotlib inline


SEED_VALUE = 0
os.environ["PYHTONSEED"] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)

alpha = 1
start_N = 30
worst_N = 500
training_iter = 1000
target = "measured log solubility in mols per litre"
target_value = 1.0

In [2]:
import datetime
def timestamp():
    time_cur = datetime.datetime.now()
    print("datetime : ", time_cur.strftime("%y%m/%d/%H:%M%S"))
    stamp = time_cur.strftime("%y%m%d%H%M%S")
    return stamp

stamp = timestamp()
print(stamp)

datetime :  2408/01/22:2859
240801222859


In [3]:
def calc_grad(y, x):
    
    """
    Args:
        y : target variables (2n,)
        x : input varibales (2n, n)

    Return:
        grad : (n)
    """
    grad_list = []
    
    for i in range(x.shape[1]):
        grad = (y[2*i] - y[2*i+1])/(x[2*i, i] - x[2*i+1, i])
        grad_list.append(grad)
        
    assert x.shape[1] == torch.tensor(grad_list).shape[0], f'{x.shape[1]}→{torch.tensor(grad_list).shape[0]}'
        
    return torch.tensor(grad_list)

In [4]:
def prepare_for_grad(x, delta=1e-3):
    
    """
    Args:
        x : input(n,)
        delta : small amount
        
    Return:
        prepare_for_grad_x) : data for calculate grads (2n, n)
    """
    
    x_for_grad_list = []
    
    for i in range(x.shape[0]):
        tmp = torch.vstack((x, x))
        tmp[0,i] += delta
        tmp[1,i] -= delta
        x_for_grad_list.append(tmp)
        
    prepare_for_grad_x = torch.cat(x_for_grad_list, dim=0)
        
    assert x.shape[0]*2 == prepare_for_grad_x.shape[0], f'{x.shape[0]*2}→{prepare_for_grad_x.shape[0]}'
    
    return prepare_for_grad_x

In [5]:
def GP_IG(model, x, N=100, sampling=100):
    
    """
    Args:
        model : trained gaussian process model
        x : input varibales (n)
        N : the number of steps for calculate integrated gradients
        samping : the number of functions sampled from model

    Return:
        all_IG : calculated all integrated gradients
    """
    
    for s in range(sampling):
        
        for n in range(N+1):
            x_n = x * n/N
            x_n_for_grad = prepare_for_grad(x_n)
            
            if n == 0:
                all_x_n_for_grad = x_n_for_grad.detach().clone()
            else:
                all_x_n_for_grad = torch.vstack((all_x_n_for_grad, x_n_for_grad))
                
        assert all_x_n_for_grad.cpu().numpy().shape == (x.shape[0]*2*(N+1) , x.shape[0]), f"{all_x_n_for_grad.numpy().shape}→{(x.shape[0]*2*(N+1) , x.shape[0])}"
        
        all_y_sampling_for_grad = model(all_x_n_for_grad).sample()
        
        for n2 in range(N+1):
            
            y_sampling_for_grad = all_y_sampling_for_grad[n2*x.shape[0]*2:((n2+1)*x.shape[0]*2)]
            x_n_for_grad = all_x_n_for_grad[n2*x.shape[0]*2:((n2+1)*x.shape[0]*2),:]

            grad = calc_grad(y_sampling_for_grad, x_n_for_grad).detach()
            
            if n2 == 0:
                all_grad = grad.detach().clone()
            else:
                all_grad = torch.vstack((all_grad, grad))

        IG = all_grad.mean(axis=0).detach()
        
        if s == 0:
            all_IG = IG.detach().clone()
        else:
            all_IG = torch.vstack((all_IG, IG))
        
    assert all_IG.cpu().numpy().shape == (sampling, x.shape[0]), f"{all_IG.numpy().shape }→{(sampling, x.shape[0])}"
    
    return all_IG

In [6]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem,Draw, Descriptors, Descriptors3D
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import rdMolDescriptors
from rdkit import DataStructs
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

def draw_mol_ig_with_2c(mol, bits_ig, alpha=2, color=0.5):
    
    bits = [int(k) for k in bits_ig.keys()]
    value_max = [v.mean(axis=0) + alpha * v.std(axis=0, ddof=0) for v in bits_ig.values()]
    value_min = [v.mean(axis=0) - alpha * v.std(axis=0, ddof=0) for v in bits_ig.values()]
    
    bits_ig_max = dict(zip(bits, value_max))
    bits_ig_min = dict(zip(bits, value_min))
    
    bitI_morgan = {}
    fp_morgan = AllChem.GetMorganFingerprint(mol, 2, bitInfo=bitI_morgan)

    bit_list = list(set(bits_ig_max.keys())&set(bitI_morgan.keys()))
    
    Ai_max_list = np.zeros(mol.GetNumAtoms())
    Ai_min_list = np.zeros(mol.GetNumAtoms())

    for bit in bit_list:

        Cn_max = bits_ig_max[int(bit)]
        Cn_min = bits_ig_min[int(bit)]

        fn = len(bitI_morgan[bit])

        for part in bitI_morgan[bit]:
            if part[1]==0:
                i = part[0]
                xn = 1
                
                Ai_max_list[i] += Cn_max / fn / xn
                Ai_min_list[i] += Cn_min / fn / xn
                

            else:
                amap={}
                env = Chem.FindAtomEnvironmentOfRadiusN(mol,
                                                        radius=part[1],
                                                        rootedAtAtom=part[0])
                submol=Chem.PathToSubmol(mol,env,atomMap=amap)

                xn = len(list(amap.keys()))

                for i in amap.keys():
                    
                    Ai_max_list[i] += Cn_max / fn / xn
                    Ai_min_list[i] += Cn_min / fn / xn
            
    scale_value = max(abs(Ai_max_list).max(), abs(Ai_min_list).max())
    Ai_max_list = (Ai_max_list / scale_value) *0.5
    Ai_min_list = (Ai_min_list / scale_value) *0.5
    
    atoms = [i for i in range(len(Ai_max_list))]
    atom_colors = dict()
    raddi_high_lights = dict()
    for i in atoms:
        raddi_high_lights[i] = color
        if Ai_max_list[i] > 0:
            if Ai_min_list[i] > 0:
                atom_colors[i] = [(1,1-Ai_max_list[i],1-Ai_max_list[i]), (1,1-Ai_min_list[i],1-Ai_min_list[i])]
            else:
                atom_colors[i] = [(1,1-Ai_max_list[i],1-Ai_max_list[i]), (1+Ai_min_list[i],1+Ai_min_list[i],1)]
        elif Ai_max_list[i] < 0:
            if Ai_min_list[i] > 0:
                atom_colors[i] = [(1+Ai_max_list[i],1+Ai_max_list[i],1), (1,1-Ai_max_list[i],1-Ai_min_list[i])]
            else:
                atom_colors[i] = [(1+Ai_max_list[i],1+Ai_max_list[i],1), (1+Ai_min_list[i],1+Ai_min_list[i],1)]
        
                

    view = rdMolDraw2D.MolDraw2DSVG(300,350)
    tm = rdMolDraw2D.PrepareMolForDrawing(mol)
    
    view.DrawMoleculeWithHighlights(tm, 
                                   '',
                                   dict(atom_colors),
                                   dict() ,
                                   raddi_high_lights,
                                   {})

    
    view.FinishDrawing()
    svg = view.GetDrawingText()

    return SVG(svg)

RDKit WARNING: [22:28:59] Enabling RDKit 2019.09.3 jupyter extensions
[22:28:59] Enabling RDKit 2019.09.3 jupyter extensions


In [ ]:
import pandas as pd

df_known = pd.read_csv("data/start_data.csv", index_col=0)
print(df_known.shape)
df_known.head()

In [ ]:
bits = [int(i) for i in df_known.iloc[:,1:].columns.to_list()]

In [ ]:
df_smiles = pd.read_csv("data/delaney-processed.csv", index_col=0)
df = pd.read_csv("data/all_data.csv", index_col=0)
df_unknown = df.drop(df_known.index)

In [11]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [13]:
results_path = f'results/results_BO_alpha_{alpha}_{stamp}'
os.makedirs(results_path, exist_ok=True)

In [16]:
%%time
import joblib

max_list = []
value_list = []
iter_number = 0

while max(df_known[target]) < target_value:
    
    df_0 = df_known.copy()
    
    iter_number += 1
    print(f"i={iter_number}, max={max(df_known[target])}")
    max_list.append(max(df_known[target]))
    
    y_known_tensor = torch.from_numpy(df_0.iloc[:,0].values.astype(np.float32))
    x_known_tensor = torch.from_numpy(df_0.iloc[:,1:].values.astype(np.float32))
    x_unknown_tensor = torch.from_numpy(df_unknown.iloc[:,1:].values.astype(np.float32))

    x_mean_tensor = x_known_tensor.mean(axis=0)
    x_std_tensor = x_known_tensor.std(axis=0)
    y_mean_tensor = y_known_tensor.mean(axis=0)
    y_std_tensor = y_known_tensor.std(axis=0)

    autoscaled_x_known_tensor = (x_known_tensor - x_mean_tensor)/x_std_tensor
    autoscaled_x_unknown_tensor = (x_unknown_tensor - x_mean_tensor)/x_std_tensor
    autoscaled_y_known_tensor = (y_known_tensor - y_mean_tensor)/y_std_tensor
    
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(autoscaled_x_known_tensor, autoscaled_y_known_tensor, likelihood)
    
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1) 
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3, min_lr=1e-4)

    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    
    for _ in range(training_iter):
        optimizer.zero_grad()
        output = model(autoscaled_x_known_tensor)
        loss = -mll(output, autoscaled_y_known_tensor)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)

    model.eval()
    likelihood.eval()
    
    print('outputscale: %.3f lengthscale: %.3f   noise: %.3f' % (model.covar_module.outputscale.item(), 
                                                                 model.covar_module.base_kernel.lengthscale.item(),
                                                                     model.likelihood.noise.item()))

    with torch.no_grad():
        autoscaled_y_pred = likelihood(model(autoscaled_x_unknown_tensor))
        y_pred = autoscaled_y_pred.mean * y_std_tensor + y_mean_tensor
        y_std = autoscaled_y_pred.stddev * y_std_tensor
        UCB = y_pred + alpha * y_std
    
    next_sample_number = torch.where(UCB == max(UCB))[0][0].item()   
    next_sample = df_unknown.iloc[[next_sample_number],:]
    print(f"mean:{y_pred[next_sample_number]:.3f}, std:{y_std[next_sample_number]:.3f}")
    print(f"{next_sample.index[0]} measured value: {df.loc[next_sample.index,target][0]:.3f}")
    value_list.append(df.loc[next_sample.index,target][0])
    
    mol = Chem.MolFromSmiles(df_smiles.loc[next_sample.index[0],"smiles"])
    gp_ig = GP_IG(model, autoscaled_x_unknown_tensor[next_sample_number,:], sampling=10)
    bits_ig = dict(zip(bits, gp_ig.numpy().T))
    
    save_dic = {}
    
    save_dic["iter_number"] = iter_number
    save_dic["x"] = autoscaled_x_unknown_tensor[next_sample_number]
    save_dic["index"] = next_sample.index[0]
    save_dic["target"] = df.loc[next_sample.index,target][0]
    save_dic["GP_IG"] = gp_ig
    save_dic["y_pred_mean"] = y_pred[next_sample_number]
    save_dic["y_pred_std"] = y_std[next_sample_number]
    
    joblib.dump(save_dic, f'{results_path}/interpret_results_{iter_number}_{next_sample.index[0]}_{stamp}.pkl')
    
    
    #display(draw_mol_ig_with_2c(mol, bits_ig, alpha=alpha))
    
    df_unknown.drop(next_sample.index.to_list(), axis=0, inplace=True)
    df_known = pd.concat([df_known, next_sample], axis=0)

i=1, max=-3.22
outputscale: 0.515 lengthscale: 2.256   noise: 0.441
mean:-4.103, std:1.286
Methylcyclohexane  measured value: -3.850
i=2, max=-3.22
outputscale: 0.909 lengthscale: 2.027   noise: 0.044
mean:-3.962, std:1.213
Terbutryn measured value: -4.000
i=3, max=-3.22
outputscale: 0.906 lengthscale: 2.146   noise: 0.043
mean:-4.091, std:1.253
2,4-Dinitrotoluene measured value: -2.820
i=4, max=-2.82
outputscale: 0.905 lengthscale: 2.269   noise: 0.028
mean:-3.881, std:1.142
p-Nitrotoluene measured value: -2.490
i=5, max=-2.49
outputscale: 0.915 lengthscale: 2.670   noise: 0.025
mean:-3.072, std:0.958
o-Nitrotoluene measured value: -2.330
i=6, max=-2.33
outputscale: 0.904 lengthscale: 2.957   noise: 0.020
mean:-3.200, std:1.131
4-Chlorotoluene measured value: -3.080
i=7, max=-2.33
outputscale: 0.929 lengthscale: 3.328   noise: 0.018
mean:-3.204, std:1.128
1,2-Dinitrobenzene measured value: -3.100
i=8, max=-2.33
outputscale: 0.937 lengthscale: 3.537   noise: 0.017
mean:-3.250, std:1.19